# Постановка задачи
$$\underset{\| x \|^2 = 1}{min} f(x) = x^T A x + b^T x$$

# Применяется метод наискорейшего спуска
$$x^{k+1} = x^k - \gamma^k \nabla f(x^k)$$
$$\gamma^k = \underset{\gamma \geq 0}{argmin} f(P_S(x^k - \gamma \nabla f(x^k)))$$

# Вычислим шаг 
$$\underset{\gamma \geq 0}{argmin} \left(\frac{x^k - \gamma(Ax^x + b)^T}{\| x^k - \gamma(Ax^x + b) \|}\right) A \left(\frac{x^k - \gamma(Ax^x + b)}{\| x^k - \gamma(Ax^x + b) \|}\right) + b^T \left(\frac{x^k - \gamma(Ax^x + b)}{\| x^k - \gamma(Ax^x + b) \|}\right)$$

In [20]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [21]:
A = np.array([[1., 1., 1.], [1., 100., 1.], [1., 1., 1.]])
b = np.array([1., 2., 3.])
print A
print b
r = 100.

[[   1.    1.    1.]
 [   1.  100.    1.]
 [   1.    1.    1.]]
[ 1.  2.  3.]


# Применим метод штрафных функций:
$$M = \left(\frac{x^k - \gamma(Ax^x + b)^T}{\| x^k - \gamma(Ax^x + b) \|}\right) A \left(\frac{x^k - \gamma(Ax^x + b)}{\| x^k - \gamma(Ax^x + b) \|}\right) + b^T \left(\frac{x^k - \gamma(Ax^x + b)}{\| x^k - \gamma(Ax^x + b) \|}\right) + r (-\gamma)^2_+$$


In [22]:
def M(gamma):
    return (xk - gamma * (np.dot(A, xk) + b)).dot(np.dot(A, xk - gamma * (np.dot(A, xk) + b))) / np.linalg.norm(xk - gamma * (np.dot(A, xk) + b)) ** 2 + b.dot((xk - gamma * (np.dot(A, xk) + b))) / np.linalg.norm(xk - gamma * (np.dot(A, xk) + b)) + r * np.max([0., -gamma]) ** 2
    

## Минимум оштрафованной функции будем находить градиентным спуском. Градиент:

$$M'_{\gamma} = \frac{\gamma^2 (-2a_2'a_3 + 2a_2 a_3') + \gamma (2a_3a_1' - 2 a_2a_1' - 2a_3' a_1) + 2a_1a_2'}{(\gamma^2 a_3' - 2 \gamma a_2' + a_1' )^2} - 2b^Tx^k \frac{\sum_{i=1}^{n}(x^k_i - \gamma(Ax^k + b)_i)(Ax^k + b)_i}{\| x^k - \gamma(Ax^k + b) \|^3} - \frac{b^T (Ax^k+b)}{\| x^k - \gamma(Ax^k + b) \|}  + \gamma(Ax^k+b)^Tb \frac{\sum_{i=1}^{n}(x^k_i - \gamma(Ax^k + b)_i)(Ax^k + b)_i}{\| x^k - \gamma(Ax^k + b) \|^3} - 2r(-\gamma)_+$$
Где введены обозначения:

$a_3 = (Ax^k + b)^T A (Ax^k + b)$

$a_2 = (x^k)^TA(Ax^k + b)$

$a_1 = (x^k)^T A x^k$

$a_3' = (Ax^k + b)^T  (Ax^k + b)$

$a_2' = (x^k)^T(Ax^k + b)$

$a_1' = (x^k)^T  x^k$


In [23]:
def gradM(gamma):
    """
    a3 = (np.dot(A, xk) + b).dot(np.dot(A, np.dot(A, xk) + b))
    a2 = (xk).dot(np.dot(A, np.dot(A, xk) + b))
    a1 = (xk).dot(np.dot(A,xk))
    a3t = (np.dot(A, xk) + b).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
    a2t = (xk).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
    a1t = (xk).dot(np.dot(np.eye(A.shape[0]),xk))
    """
    first = (gamma ** 2 *(-2 * a2t * a3 + 2 * a2 * a3t) + gamma * (2 * a3 * a1t - 2 * a2 * a1t - 2 * a3t * a1) + 2 * a1 * a2t) / ((gamma ** 2 * a3t - 2 * gamma * a2t + a1t)**2)
    sumfrac = sum((xk - gamma * (np.dot(A, xk) + b)) * (np.dot(A, xk) + b)) / (np.linalg.norm(xk - gamma * (np.dot(A, xk) + b)) ** 3)
    second  = - 2 * np.dot(b, xk) * sumfrac
    third = - (np.dot(b, np.dot(A, xk) + b)) / (np.linalg.norm(xk - gamma * (np.dot(A, xk ) + b)))
    fourth =  gamma * np.dot(np.dot(A, xk) + b,b) * sumfrac
    fifth = - 2 * r * max([0., -gamma])
    return first + second + third + fourth + fifth

# Поскольку вычисление шага как в методе наискорешего спуска немного затруднительно, а константа Липшица для градиента не видна, применим для вычисления шага backtracking line search:

In [46]:
def backtracking_line_search(f, gradf, x, p): # attention! this is for scalar function of one varianle
    alpha = 1.
    c = 1e-3
    ro = 0.5
    i = 0
    while(f(x + alpha * p) > (f(x) +  c * alpha * gradf(x) * p)):
        alpha = ro * alpha
        print 'iteration number: ', i
        print 'alpha_i = ', alpha
        i += 1
    return alpha

In [71]:
def get_new_gamma(f, gradf, x0):
    #Pts_x = [x0]
    #Pts_y = [f(x0)]
    p = lambda x: -gradf(x)
    x1 = x0
    x2 = x1 + backtracking_line_search(f, gradf, x1, p(x1)) * p(x1)
    j = 0
    while (np.abs(x2 - x1) > 1e-3):
        x1 = x2
        x2 = x1 + backtracking_line_search(f, gradf, x1, p(x1)) * p(x1)
        #Pts_x.append(x2)
        #Pts_y.append(f(x2))
        j += 1
    print 'total iterations: ', j
    return x2#, Pts_x, Pts_y

In [74]:
#test cell
xk = np.array([1., 0., 0])
a3 = (np.dot(A, xk) + b).dot(np.dot(A, np.dot(A, xk) + b))
a2 = (xk).dot(np.dot(A, np.dot(A, xk) + b))
a1 = (xk).dot(np.dot(A,xk))
a3t = (np.dot(A, xk) + b).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
a2t = (xk).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
a1t = (xk).dot(np.dot(np.eye(A.shape[0]),xk))
print get_new_gamma(M, gradM, 528.)

iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  2
alpha_i =  0.125
iteration number:  3
alpha_i =  0.0625
total iterations:  36
528.497099458


In [75]:
print_rate = 10
k = 0
x0 = np.array([1., 0., 1.])
prevx = x0
notenough = True
while(notenough):
    xk = x0
    a3 = (np.dot(A, xk) + b).dot(np.dot(A, np.dot(A, xk) + b))
    a2 = (xk).dot(np.dot(A, np.dot(A, xk) + b))
    a1 = (xk).dot(np.dot(A,xk))
    a3t = (np.dot(A, xk) + b).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
    a2t = (xk).dot(np.dot(np.eye(A.shape[0]), np.dot(A, xk) + b))
    a1t = (xk).dot(np.dot(np.eye(A.shape[0]),xk))
    gamma = get_new_gamma(M, gradM, 500.0)
    xkk = (xk - gamma * gradM(gamma)) / np.linalg.norm(xk - gamma * gradM(gamma))
    if (np.linalg.norm(xkk - xk) < 1e-4):
        notenough = False
    k += 1
    if (k % print_rate == 0):
        print 'iter_num = ', k
print xkk

iteration number:  0
alpha_i =  0.5
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  2
alpha_i =  0.125
iteration number:  3
alpha_i =  0.0625
iteration number:  4
alpha_i =  0.03125
iteration number:  5
alpha_i =  0.015625
total iterations:  110287
iter_num =  1
iteration number:  0
alpha_i =  0.5
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  2
alpha_i =  0.125
iteration number:  3
alpha_i =  0.0625
iteration number:  4
alpha_i =  0.03125
iteration number:  5
alpha_i =  0.015625
total iterations:  110287
iter_num =  2
iteration number:  0
alpha_i =  0.5
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  0
alpha_i =  0.5
iteration number:  1
alpha_i =  0.25
iteration number:  2
alpha_i =  0.125
iteration number:  3

KeyboardInterrupt: 

In [77]:
print xkk
print xkk.dot(np.dot(A, xkk)) + np.dot(b, xkk)

[ 0.60028747  0.52849779  0.60028747]
34.0995191591
